In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime as dt
import os
import json

%matplotlib inline

In [2]:
def concat_dataframes(path, df_type):
    dfs_list = []
    dfs_rows_len_list = []
    
    for user in os.listdir(path):
        for file in os.listdir(os.path.join(path, user)):
            if file.find(df_type) != -1:
                df = pd.read_csv(os.path.join(path, user, file))
                
                if df_type != 'broadcasts':
                    df = df.drop(["timestamp"], axis=1)
#                 df = (df - df.min()) / (df.max() - df.min())
                
                df["user"] = int(user.split('_')[1])
                
                dfs_list.append(df)
    
    return pd.concat(dfs_list, ignore_index=True)


def get_broadcasts_dataframe(data_path, window_type, window_size):
    return concat_dataframes(os.path.join(data_path, window_type, window_size), 'broadcasts'), create_file_for_results('broadcasts')


def get_broadcasts_list(file):
    with open(file, 'r') as f:
        lines = f.readlines()
    
    return [x.replace('\n', '') for x in lines]


def create_file_for_results(data_type):
    res_folder = '.\\_results'
    if os.path.exists(res_folder) is False:
        os.makedirs(res_folder)
    
    file = os.path.join(res_folder, data_type + '_results.json')
    if os.path.exists(file) is False:
        with open(file, 'w') as f:
            json.dump({'stub': None}, f)
        
    return file    


def update_file_with_results(file_path, results_dict):
    import collections.abc

    def update(d, u):
        for k, v in u.items():
            if isinstance(v, collections.abc.Mapping):
                d[k] = update(d.get(k, {}), v)
            else:
                d[k] = v
        return d
    
    with open(file_path, 'r') as f:
        res = json.load(f)
    
    res = update(res, results_dict)
    
    with open(file_path, 'w') as f:
        json.dump(res, f, sort_keys=True, indent=2)
        
        
def get_dict_with_results(json_path):
    with open(json_path, 'r') as f:
        res = json.load(f)
    return res    


def get_cv_split(X, y, group_labels, valid_user_label):
    predefined_split_array = np.zeros(group_labels.shape[0])
    i = 0
    test_array = [x for x in range(group_labels.shape[0])]
    for test, _ in LeaveOneGroupOut().split(X, y, group_labels):
        diff = np.setdiff1d(test_array, test)
        if np.all(group_labels[diff[0] : diff[-1]] == valid_user_label) is np.bool_(True):
            for sample in diff:
                predefined_split_array[sample] = -1
        else:
            for sample in diff:
                predefined_split_array[sample] = i
            i += 1
    return predefined_split_array


def fit_label_encoder(broadcasts_list):
    le = LabelEncoder()
    le.fit(broadcasts_list)
    return le


def prepare_dataset(df, user):
    df_ = split_users_into_two_classes(df.copy(), user)
    
    group_labels = df_.labels.to_numpy().copy()
    df_ = df_.drop(['labels', 'timestamp'], axis=1)
    
    dataset = df_.to_numpy().copy()
    X = dataset[:, :-1]
    y = dataset[:, -1]
    
    return X, y, group_labels


def split_users_into_two_classes(df, valid_user_label):
    df.loc[df["user"] != valid_user_label, "user"] = 0
    df.loc[df["user"] == valid_user_label, "user"] = 1
    return df  


def generate_train_dataset(df, user, ex_user, is_SVM = False):
    df_ = df.copy()

    df_for_test = []

    df__ = df_[df_.labels == ex_user].copy()
    df_for_test.append(df__)
    df_ = df_.drop(df__.index, axis=0)

    for user_ in df_.labels.unique():
        if user_ != ex_user:
            test_size = int((0.25 * df_[df_.labels == user_].shape[0]) - 1)
            df__ = df_[df_.labels == user_].sample(test_size).copy()
            df_for_test.append(df__)
            df_ = df_.drop(df__.index, axis=0)

    df_ = split_users_into_two_classes(df_.copy(), user)
    
    valid_user_count = df_[df_.user == 1].shape[0]
    intruder_count = df_[df_.user != 1].shape[0]
    
    if valid_user_count < intruder_count:
        df_ = df_.drop(df_[df_.user != 1].sample(intruder_count - valid_user_count).index)
    else:
        df_ = df_.drop(df_[df_.user == 1].sample(valid_user_count - intruder_count).index)
          
    if is_SVM:    
        df_.loc[df_.user == 0, 'user'] = -1

    df_ = df_.drop(["labels", 'timestamp'], axis=1)

    dataset = df_.to_numpy().copy()
    np.random.shuffle(dataset)
    
    X = dataset[:, 0:-1]
    y = dataset[:, -1]
    
    return X, y, df_for_test


def generate_test_dataset(df_list, user, ex_user, is_SVM = False):
    test_df = pd.concat(df_list)

    valid_user_in_test_count = test_df[test_df.labels == user].shape[0]
    ex_user_in_test_count = test_df[test_df.labels == ex_user].shape[0]
    others_in_test_count = [test_df[test_df.labels == x].shape[0]
                            for x in test_df.labels.unique() if x != user and x != ex_user]

    others_test_count = sum(others_in_test_count)
    part_size = min(valid_user_in_test_count, ex_user_in_test_count)
    if others_test_count <= min(valid_user_in_test_count, ex_user_in_test_count):
        part_size = others_test_count    
        
    new_df_parts = []    

    new_df_parts.append(test_df[test_df.labels == user].sample(part_size).copy())
    new_df_parts.append(test_df[test_df.labels == ex_user].sample(part_size).copy())
    new_df_parts.append(test_df[~test_df.labels.isin([user, ex_user])].sample(part_size).copy())
    
    test_df = pd.concat(new_df_parts)
    
    test_df.loc[test_df.labels == user, "user"] = 1
    if is_SVM:
        test_df.loc[test_df.labels != user, "user"] = -1
    else:
        test_df.loc[test_df.labels != user, "user"] = 0

    print("True: ", test_df[test_df.user == 1].shape)
    print("Shape: ", test_df.shape)
    for x in test_df.labels.unique():
        print("Count ", x, ": ", test_df[test_df.labels == x].shape)

    test_df = test_df.drop("labels", axis=1)

    test_dataset = test_df.to_numpy().copy()
    
    timestamps = test_dataset[:, 0].copy()
    X_test = test_dataset[:, 1:-1].copy()
    y_test = test_dataset[:, -1].copy()

    return X_test, [int(x) for x in y_test], timestamps

In [3]:
def broadcasts_model_cross_validation(results_file, model, df, model_tag, df_type, window_type, window_size):
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")

        X, y, group_labels = prepare_dataset(df, user)

        cv_split = PredefinedSplit(test_fold=get_cv_split(X, y, group_labels, user))
        scoring = ('accuracy')

        cv_results = cross_validate(model, X, y, scoring=scoring, cv=cv_split, n_jobs=-1)
        accuracy = cv_results['test_score']
        
        results = {
            df_type: {
                window_type: {
                    window_size: {
                        model_tag: {
                            "cross_validation": {
                                "valid_user": {
                                    str(user): {
                                        "accuracy": accuracy.tolist(),
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        
        update_file_with_results(results_file, results)

        print("CV accuracy list: ", accuracy)
        print("CV mean accuracy: ", np.mean(accuracy))
        print("CV min accuracy: ", np.min(accuracy))
        print("CV max accuracy: ", np.max(accuracy))

        print("--------------------------------------------------------------------------------")
        
        
def broadcasts_model_final_validation(results_file, model, df, model_tag, df_type, window_type, window_size, is_SVM = False):
    for user in df.labels.unique():
        print("Valid User: ", user)
        print("--------------------------------------------------------------------------------")
        for ex_user in df.labels.unique():
            if ex_user != user:
                X, y, df_for_test = generate_train_dataset(df, user, ex_user, is_SVM)
                
                model.fit(X, y)

                X_test, y_test, timestamps = generate_test_dataset(df_for_test, user, ex_user, is_SVM)

                predict = model.predict(X_test)
                if is_SVM:
                    proba = model.decision_function(X_test)
                else:
                    proba = model.predict_proba(X_test)

                results = {
                    df_type: {
                        window_type: {
                            window_size: {
                                model_tag: {
                                    "final_validation": {
                                        "valid_user": {
                                            str(user): {
                                                "extracted_user": {
                                                    str(ex_user): {
                                                        "test": y_test,
                                                        "predict": predict.tolist(),
                                                        "proba": proba.tolist(),
                                                        "time": timestamps.tolist()
                                                    }
                                                }
                                            }
                                        }
                                    }
                                }
                            }
                        }
                    }
                }

                update_file_with_results(results_file, results)

                print("Valid user = ", user, ", Extracted user = ", ex_user, "accuracy = ", accuracy_score(y_test, predict))
                print("--------------------------------------------------------------------------------")     

### Learning settings
### ***

In [28]:
DATA_PATH = '..\\..\\scripts\\_features_all'

DATA_TYPE = "broadcasts"

WINDOW_TYPE = "rolling"
WINDOW_SIZE = "60s"

BROADCASTS_LIST_FILE = "broadcasts.list"

### ***

In [33]:
df, RESULTS_FILE = get_broadcasts_dataframe(DATA_PATH, WINDOW_TYPE, WINDOW_SIZE)

In [34]:
broadcasts_list = get_broadcasts_list(BROADCASTS_LIST_FILE)
le = fit_label_encoder(broadcasts_list)

In [35]:
df['action'] = le.transform(df.action)
df['labels'] = df['user']

## CatBoostClassifier CV

In [8]:
model_params = {
    'iterations': 100,
    'depth': 6,
    'loss_function': 'Logloss',
    'l2_leaf_reg': 1,
    'leaf_estimation_iterations': 5,
    'logging_level': 'Silent'
}

model = CatBoostClassifier(**model_params)
broadcasts_model_cross_validation(RESULTS_FILE, model, df, "CatBoost", DATA_TYPE, WINDOW_TYPE, WINDOW_SIZE)

Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [1.         1.         0.97938658 1.         1.         1.
 1.        ]
CV mean accuracy:  0.9970552263794721
CV min accuracy:  0.9793865846563046
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [1. 1. 1. 1. 1. 1. 1.]
CV mean accuracy:  1.0
CV min accuracy:  1.0
CV max accuracy:  1.0
--------------------------------------------------------------------------------
Valid User:  4
------------------------------------------------

## CatBoostClassifier Final Validation

In [9]:
model_params = {
    'iterations': 100,
    'depth': 6,
    'loss_function': 'Logloss',
    'l2_leaf_reg': 1,
    'leaf_estimation_iterations': 5,
    'logging_level': 'Silent'
}

model = CatBoostClassifier(**model_params)
broadcasts_model_final_validation(RESULTS_FILE, model, df, "CatBoost", DATA_TYPE, WINDOW_TYPE, WINDOW_SIZE)

Valid User:  1
--------------------------------------------------------------------------------
True:  (3471, 4)
Shape:  (10413, 4)
Count  1 :  (3471, 4)
Count  2 :  (3471, 4)
Count  7 :  (131, 4)
Count  3 :  (267, 4)
Count  6 :  (1162, 4)
Count  5 :  (1107, 4)
Count  4 :  (590, 4)
Count  8 :  (214, 4)
Valid user =  1 , Extracted user =  2 accuracy =  0.4711418419283588
--------------------------------------------------------------------------------
True:  (3471, 4)
Shape:  (10413, 4)
Count  1 :  (3471, 4)
Count  3 :  (3471, 4)
Count  6 :  (1194, 4)
Count  4 :  (625, 4)
Count  5 :  (1229, 4)
Count  2 :  (118, 4)
Count  7 :  (125, 4)
Count  8 :  (180, 4)
Valid user =  1 , Extracted user =  3 accuracy =  0.4985114760395659
--------------------------------------------------------------------------------
True:  (3471, 4)
Shape:  (10413, 4)
Count  1 :  (3471, 4)
Count  4 :  (3471, 4)
Count  6 :  (1321, 4)
Count  5 :  (1375, 4)
Count  2 :  (123, 4)
Count  8 :  (237, 4)
Count  7 :  (142, 4)
C

True:  (4387, 4)
Shape:  (13161, 4)
Count  4 :  (4387, 4)
Count  2 :  (4387, 4)
Count  5 :  (1558, 4)
Count  1 :  (476, 4)
Count  8 :  (231, 4)
Count  6 :  (1596, 4)
Count  7 :  (176, 4)
Count  3 :  (350, 4)
Valid user =  4 , Extracted user =  2 accuracy =  0.5272395714611352
--------------------------------------------------------------------------------
True:  (6014, 4)
Shape:  (18042, 4)
Count  4 :  (6014, 4)
Count  3 :  (6014, 4)
Count  6 :  (2238, 4)
Count  5 :  (2277, 4)
Count  8 :  (402, 4)
Count  1 :  (651, 4)
Count  7 :  (225, 4)
Count  2 :  (221, 4)
Valid user =  4 , Extracted user =  3 accuracy =  0.4646934929608691
--------------------------------------------------------------------------------
True:  (6014, 4)
Shape:  (18042, 4)
Count  4 :  (6014, 4)
Count  5 :  (6014, 4)
Count  6 :  (3198, 4)
Count  8 :  (515, 4)
Count  1 :  (901, 4)
Count  2 :  (289, 4)
Count  7 :  (349, 4)
Count  3 :  (762, 4)
Valid user =  4 , Extracted user =  5 accuracy =  0.56168939142002
----------

Valid user =  7 , Extracted user =  3 accuracy =  0.5648655703471679
--------------------------------------------------------------------------------
True:  (1277, 4)
Shape:  (3831, 4)
Count  7 :  (1277, 4)
Count  4 :  (1277, 4)
Count  1 :  (133, 4)
Count  6 :  (439, 4)
Count  5 :  (471, 4)
Count  3 :  (104, 4)
Count  8 :  (85, 4)
Count  2 :  (45, 4)
Valid user =  7 , Extracted user =  4 accuracy =  0.6606630122683372
--------------------------------------------------------------------------------
True:  (1277, 4)
Shape:  (3831, 4)
Count  7 :  (1277, 4)
Count  5 :  (1277, 4)
Count  6 :  (581, 4)
Count  4 :  (276, 4)
Count  3 :  (119, 4)
Count  8 :  (85, 4)
Count  2 :  (54, 4)
Count  1 :  (162, 4)
Valid user =  7 , Extracted user =  5 accuracy =  0.5468546071521796
--------------------------------------------------------------------------------
True:  (1277, 4)
Shape:  (3831, 4)
Count  7 :  (1277, 4)
Count  6 :  (1277, 4)
Count  3 :  (154, 4)
Count  5 :  (549, 4)
Count  8 :  (87, 4)
Cou

## CategoricalNB CV

In [10]:
model_params = {
    'alpha': 0,
    'fit_prior': False,
    'class_prior': [0.5, 0.5],
    'min_categories': len(le.classes_)
}

model = CategoricalNB(**model_params)
broadcasts_model_cross_validation(RESULTS_FILE, model, df, "NB", DATA_TYPE, WINDOW_TYPE, WINDOW_SIZE)

Valid User:  1
--------------------------------------------------------------------------------
CV accuracy list:  [0.15272396 0.12881608 0.20226914 0.24078128 0.9293741  0.27624633
 0.17984805]
CV mean accuracy:  0.3014369912751826
CV min accuracy:  0.12881608339538347
CV max accuracy:  0.9293740988889831
--------------------------------------------------------------------------------
Valid User:  2
--------------------------------------------------------------------------------
CV accuracy list:  [0.32262942 0.31971333 0.464259   0.48299377 0.91656772 0.42072336
 0.32432432]
CV mean accuracy:  0.464458703292128
CV min accuracy:  0.3197133283693224
CV max accuracy:  0.9165677211432448
--------------------------------------------------------------------------------
Valid User:  3
--------------------------------------------------------------------------------
CV accuracy list:  [0.50428397 0.54137224 0.42531793 0.53108688 0.94572131 0.50381232
 0.51949184]
CV mean accuracy:  0.56729806

## CategoricalNB Final Validation

In [ ]:
model_params = {
    'min_categories': len(le.classes_)
}

model = CategoricalNB(**model_params)
broadcasts_model_final_validation(RESULTS_FILE, model, df, "NB", DATA_TYPE, WINDOW_TYPE, WINDOW_SIZE)

Valid User:  1
--------------------------------------------------------------------------------
True:  (3471, 4)
Shape:  (10413, 4)
Count  1 :  (3471, 4)
Count  2 :  (3471, 4)
Count  6 :  (1169, 4)
Count  5 :  (1128, 4)
Count  4 :  (577, 4)
Count  3 :  (255, 4)
Count  7 :  (124, 4)
Count  8 :  (218, 4)
Valid user =  1 , Extracted user =  2 accuracy =  0.6668587342744646
--------------------------------------------------------------------------------
True:  (3471, 4)
Shape:  (10413, 4)
Count  1 :  (3471, 4)
Count  3 :  (3471, 4)
Count  6 :  (1204, 4)
Count  5 :  (1209, 4)
Count  4 :  (617, 4)
Count  8 :  (205, 4)
Count  2 :  (109, 4)
Count  7 :  (127, 4)
Valid user =  1 , Extracted user =  3 accuracy =  0.6666666666666666
--------------------------------------------------------------------------------
True:  (3471, 4)
Shape:  (10413, 4)
Count  1 :  (3471, 4)
Count  4 :  (3471, 4)
Count  8 :  (222, 4)
Count  5 :  (1358, 4)
Count  6 :  (1299, 4)
Count  2 :  (153, 4)
Count  3 :  (302, 4)
C